In [1]:
import numpy as np
from ler.rates import LeR

In [2]:
ler = LeR(verbose=False, )

In [3]:
ler.batch_size = 50000
lensed_params = ler.lensed_cbc_statistics(size=1000000, resume=True)

lensed params will be store in ./lensed_param.json
chosen batch size = 50000 with total size = 1000000
There will be 20 batche(s)
resuming from ./lensed_param.json
Batch no. 10
sampling lensed params...
solving lens equations...


100%|███████████████████████████████████████████████████████| 50000/50000 [00:14<00:00, 3526.24it/s]


calculating snrs...
Batch no. 11
sampling lensed params...
solving lens equations...


100%|███████████████████████████████████████████████████████| 50000/50000 [00:15<00:00, 3180.80it/s]


calculating snrs...
Batch no. 12
sampling lensed params...
solving lens equations...


...

calculating snrs...
Batch no. 19
sampling lensed params...
solving lens equations...


100%|███████████████████████████████████████████████████████| 50000/50000 [00:14<00:00, 3420.59it/s]


calculating snrs...
Batch no. 20
sampling lensed params...
solving lens equations...


100%|███████████████████████████████████████████████████████| 50000/50000 [00:14<00:00, 3484.03it/s]


calculating snrs...


In [4]:
ler.lensed_rate();

getting lensed_params from json file ./lensed_param.json...
total lensed rate (yr^-1) (with step function): 1.1304423159873103
number of simulated lensed detectable events: 4549
number of simulated all lensed events: 1000000
storing detectable lensed params in ./lensed_param_detectable.json


In [8]:
ler.lensed_rate(snr_threshold=8, num_img=4);

getting lensed_params from json file ./lensed_param.json...
total lensed rate (yr^-1) (with step function): 0.08697621688185504
number of simulated lensed detectable events: 350
number of simulated all lensed events: 1000000
storing detectable lensed params in ./lensed_param_detectable.json


In [9]:
(350)/4549*1.13

0.0869421850956254

In [3]:
ler.batch_size = 50000
unlensed_params = ler.unlensed_cbc_statistics(size=100000)

unlensed params will be store in ./unlensed_param.json
chosen batch size = 50000 with total size = 100000
There will be 2 batche(s)
Batch no. 1
sampling gw source params...
calculating snrs...
Batch no. 2
sampling gw source params...
calculating snrs...


In [5]:
ler.unlensed_rate();

getting unlensed_params from json file ./unlensed_param.json...
number of simulated unlensed detectable events: 399
number of all simulated unlensed events: 100000
total unlensed rate (yr^-1) (with step function): 413.0563573223664
storing detectable unlensed params in ./unlensed_param_detectable.json


In [4]:
ler.unlensed_rate(snr_recalculation=True,);

getting unlensed_params from json file ./unlensed_param.json...


100%|████████████████████████████████████████████████████████████| 590/590 [00:03<00:00, 162.47it/s]

number of simulated unlensed detectable events: 399
number of all simulated unlensed events: 100000
total unlensed rate (yr^-1) (with step function): 413.0563573223664
storing detectable unlensed params in ./unlensed_param_detectable.json


In [5]:
test = {'a':1, 'b':2, 'c':3}

if 'a' and 'b' in test:
    print('yes')

yes


In [ ]:
ler.batch_size = 50000
lensed_params = ler.lensed_cbc_statistics(size=1000000, save_batch=False)

lensed params will be store in ./lensed_param.json
chosen batch size = 50000 with total size = 100000
There will be 2 batche(s)
Batch no. 1
sampling lensed params...
solving lens equations...


100%|███████████████████████████████████████████████████████| 50000/50000 [00:14<00:00, 3446.11it/s]


calculating snrs...
Batch no. 2
sampling lensed params...
solving lens equations...


 58%|███████████████████████████████▉                       | 29040/50000 [00:09<00:07, 2957.44it/s]


KeyboardInterrupt: 

In [24]:
ler.lensed_rate();

getting lensed_params from json file ./lensed_param.json...
total lensed rate (yr^-1) (with step function): 1.1803915148251756
number of simulated lensed detectable events: 475
number of simulated all lensed events: 100000
storing detectable lensed params in ./lensed_param_detectable.json
Time taken to load the json file: 0:00:00.000003
Time taken to append the dictionary: 0:00:00.000001
Time taken to save the json file: 0:00:00.028694
Time taken to load the json file: 0:00:00.000001
Time taken to append the dictionary: 0:00:00
Time taken to save the json file: 0:00:00.000216


In [1]:
from ler.rates import LeR
ler = LeR(verbose=False, snr_type='ann', waveform_approximant='IMRPhenomXPHM', pdet=True)

In [3]:
ler.batch_size = 50000
lensed_params = ler.unlensed_cbc_statistics(size=100000)

unlensed params will be store in ./unlensed_param.json
chosen batch size = 50000 with total size = 100000
There will be 2 batche(s)
Batch no. 1
sampling gw source params...
calculating snrs...
Batch no. 2
sampling gw source params...
calculating snrs...


In [8]:
ler.unlensed_rate();

getting unlensed_params from json file ./unlensed_param.json...
number of simulated unlensed detectable events: 564
number of all simulated unlensed events: 100000
total unlensed rate (yr^-1) (with step function): 583.869136666202
storing detectable unlensed params in ./unlensed_param_detectable.json


In [9]:
ler.unlensed_rate(snr_recalculation=True,);

getting unlensed_params from json file ./unlensed_param.json...


100%|████████████████████████████████████████████████████████████| 800/800 [00:04<00:00, 168.10it/s]

number of simulated unlensed detectable events: 471
number of all simulated unlensed events: 100000
total unlensed rate (yr^-1) (with step function): 487.592842854222
storing detectable unlensed params in ./unlensed_param_detectable.json


In [8]:
ler.batch_size = 50000
lensed_params = ler.lensed_cbc_statistics(size=100000)

lensed params will be store in ./lensed_param.json
chosen batch size = 50000 with total size = 100000
There will be 2 batche(s)
Batch no. 1
sampling lensed params...
solving lens equations...


100%|███████████████████████████████████████████████████████| 50000/50000 [00:14<00:00, 3434.90it/s]


calculating snrs...
146/146 [==============================] - 0s 296us/step
Batch no. 2
sampling lensed params...
solving lens equations...


100%|███████████████████████████████████████████████████████| 50000/50000 [00:14<00:00, 3407.38it/s]


calculating snrs...
148/148 [==============================] - 0s 303us/step


In [11]:
ler.lensed_rate(detectability_condition = "pdet");

getting lensed_params from json file ./lensed_param.json...
total lensed rate (yr^-1) (with pdet function): 1.3841643658055218
number of simulated lensed detectable events: 557
number of simulated all lensed events: 100000
storing detectable lensed params in ./lensed_param_detectable.json


In [12]:
ler.lensed_rate();

getting lensed_params from json file ./lensed_param.json...
total lensed rate (yr^-1) (with step function): 1.4612004436151647
number of simulated lensed detectable events: 588
number of simulated all lensed events: 100000
storing detectable lensed params in ./lensed_param_detectable.json


In [10]:
ler = LeR(verbose=False, waveform_approximant='IMRPhenomXPHM', snr_type='inner_product')

In [11]:
ler.batch_size = 50000
lensed_params = ler.unlensed_cbc_statistics(size=100000)

unlensed params will be store in ./unlensed_param.json
chosen batch size = 50000 with total size = 100000
There will be 2 batche(s)
Batch no. 1
sampling gw source params...
calculating snrs...
solving SNR with inner product


100%|████████████████████████████████████████████████████████| 46322/46322 [00:59<00:00, 784.11it/s]


Batch no. 2
sampling gw source params...
calculating snrs...
solving SNR with inner product


100%|████████████████████████████████████████████████████████| 46379/46379 [00:59<00:00, 784.63it/s]


In [12]:
ler.unlensed_rate();

getting unlensed_params from json file ./unlensed_param.json...
number of simulated unlensed detectable events: 464
number of all simulated unlensed events: 100000
total unlensed rate (yr^-1) (with step function): 480.3462400941805
storing detectable unlensed params in ./unlensed_param_detectable.json


In [24]:
ler.batch_size = 50000
lensed_params = ler.lensed_cbc_statistics(size=100000)

lensed params will be store in ./lensed_param.json
chosen batch size = 50000 with total size = 100000
There will be 2 batche(s)
Batch no. 1
sampling lensed params...
solving lens equations...


100%|███████████████████████████████████████████████████████| 50000/50000 [00:15<00:00, 3288.22it/s]


calculating snrs...
solving SNR with inner product


100%|████████████████████████████████████████████████████████| 44067/44067 [00:45<00:00, 958.17it/s]


solving SNR with inner product


...

100%|████████████████████████████████████████████████████████| 43866/43866 [02:32<00:00, 288.45it/s]


solving SNR with inner product


100%|██████████████████████████████████████████████████████████| 9880/9880 [00:41<00:00, 238.67it/s]


solving SNR with inner product


100%|██████████████████████████████████████████████████████████| 4602/4602 [00:24<00:00, 186.74it/s]


In [26]:
ler.lensed_rate(detectability_condition = "pdet");

getting lensed_params from json file ./lensed_param.json...
total lensed rate (yr^-1) (with pdet function): 1.0288043939739424
number of simulated lensed detectable events: 414
number of simulated all lensed events: 100000
storing detectable lensed params in ./lensed_param_detectable.json


In [25]:
ler.lensed_rate();

getting lensed_params from json file ./lensed_param.json...
total lensed rate (yr^-1) (with step function): 1.0959003327113734
number of simulated lensed detectable events: 441
number of simulated all lensed events: 100000
storing detectable lensed params in ./lensed_param_detectable.json


In [28]:
lensed_params['image_type'][100:200]

array([[1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 1., 2., 2.],
       [2., 1., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 3., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 1., 2., 2.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 1., 2., 2.],
       [1., 2., 0., 0.],
       [3., 2., 1., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 1., 2., 2.],
       [1., 1., 2., 2.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 3., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 1., 2., 2.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],


In [5]:
from ler.lens_galaxy_population import axis_ratio_rayleigh
from lenstronomy.Util.param_util import phi_q2_ellipticity

q = axis_ratio_rayleigh(sigma=np.array([160.]))[0]
phi = np.random.uniform(0.0, 2*np.pi, size=1)[0]
e1, e2 = phi_q2_ellipticity(phi, q)
# resampling power-law index and shear
gamma = np.random.normal(loc=2.0, scale=0.2, size=1)[0]
gamma1, gamma2 = np.random.normal(loc=0, scale=0.05,size=(2,1))

In [6]:
q

0.7308830987609083

In [7]:
phi

2.654184878149677

In [9]:
e2

-0.12867461019938886

In [10]:
gamma

1.9548018008560062

In [11]:
# gw parameter sampling
        if size is None:
            size = self.size

        # get json file name
        if output_jsonfile is None:
            output_jsonfile = self.json_file_names["lensed_param"]
        else:
            self.json_file_names["lensed_param"] = output_jsonfile
        print(f"lensed params will be store in {output_jsonfile}")

        # sampling in batches
        self.dict_buffer = None
        batch_handler(
            size=size,
            batch_size=self.batch_size,
            sampling_routine=self.lensed_sampling_routine,
            output_jsonfile=output_jsonfile,
            save_batch=save_batch,
            resume=resume,
        )

        if save_batch:
            lensed_param = get_param_from_json(output_jsonfile)
        else:
            lensed_param = self.dict_buffer.copy()
            # save the last batch
            append_json(output_jsonfile, lensed_param, replace=True)
        self.dict_buffer = None
        return lensed_param

array([0.0198927])

In [ ]:
# get lensed params
        print("sampling lensed params...")
        lensed_param = self.sample_lens_parameters(size=size)
        # now get (strongly lensed) image paramters along with lens parameters
        lensed_param = self.image_properties(lensed_param)
        # Get all of the signal to noise ratios
        print("calculating snrs...")
        snrs, lensed_param = self.get_lensed_snrs(
            snr_calculator=self.snr,
            list_of_detectors=self.list_of_detectors,
            lensed_param=lensed_param,
        )
        lensed_param.update(snrs)

        # adding batches
        if not save_batch:
            if self.dict_buffer is None:
                self.dict_buffer = lensed_param
            else:
                self.dict_buffer = add_dict_values(self.dict_buffer.copy(), lensed_param)
        else:
            # store all params in json file
            self.dict_buffer = append_json(file_name=output_jsonfile, new_dictionary=lensed_param,  old_dictionary=self.dict_buffer, replace=not (resume))

        return lensed_param